In [ ]:
# export
from fastai_local.core import *
from fastai_local.test import *
from fastai_local.imports import *
from fastai_local.notebook.export import *
import nbformat
from nbconvert.preprocessors import Preprocessor
from nbconvert import HTMLExporter
from traitlets.config import Config

In [ ]:
# default_exp notebook.export2html

# Converting notebooks to html

> The functions that transform the dev notebooks in the documentation of the library

## Preprocessing notebook

### Cell processors

In [ ]:
#export
def remove_widget_state(cell):
    if cell['cell_type'] == 'code' and 'outputs' in cell:
        cell['outputs'] = [l for l in cell['outputs'] 
                           if not ('data' in l and 'application/vnd.jupyter.widget-view+json' in l.data)]
    return cell

In [ ]:
#export
def hide_cells(cell):
    if check_re(cell, r'^\s*#\s*export\s+'): cell['metadata'] = {'hide_input': True}
    if check_re(cell, r'^\s*#\s*hide\s+'):   cell['metadata'] = {'hide_input': True}    
    return cell

In [ ]:
cell = {'cell_type': 'code', 'source': '# export\nfrom fastai_local.core import *'}
cell1 = hide_cells(cell.copy())
assert 'metadata' in cell1
assert 'hide_input' in cell1['metadata']
assert cell1['metadata']['hide_input']

cell = {'cell_type': 'code', 'source': '# hide\nfrom fastai_local.core import *'}
cell1 = hide_cells(cell.copy())
assert 'metadata' in cell1
assert 'hide_input' in cell1['metadata']
assert cell1['metadata']['hide_input']

cell = {'cell_type': 'code', 'source': '# exports\nfrom fastai_local.core import *'}
test_eq(cell, hide_cells(cell.copy()))

### Grabbing metada

In [ ]:
def get_metadata(cells):
    "Find the cell with title and summary in `cells`."
    pat = re.compile('^\s*#\s*([^\n]*)\n*>\s*([^\n]*)')
    for cell in cells:
        if cell['cell_type'] == 'markdown':
            match = re.match(pat, cell['source'])
            if match: return {'keywords': 'fastai',
                              'summary' : match.groups()[1],
                              'title'   : match.groups()[0]}

In [ ]:
tst_nb = read_nb('99_export.ipynb')
test_eq(get_metadata(tst_nb['cells']), {
    'keywords': 'fastai',
    'summary': 'The functions that transform the dev notebooks in the fastai library',
    'title': 'Converting notebooks to modules'})

## Conversion

In [ ]:
#export
process_cells = [remove_widget_state, hide_cells]

In [ ]:
#hide
__file__ = Path('fastai_docs')/'notebook'/'jekyll.tpl'

In [ ]:
(Path('fastai_docs')/'notebook'/'jekyll.tpl').absolute()

PosixPath('/home/ubuntu/fastai_docs/dev/fastai_docs/notebook/jekyll.tpl')

In [ ]:
class NbExporter(HTMLExporter):
    
    @property
    def template_path(self):
        return super().template_path+[str((Path('fastai_docs')/'notebook'/).absolute())]

    def _template_file_default(self):
        """
        We want to use the new template we ship with our library.
        """
        return 'test_template' # full

In [ ]:
def _exporter():
    exporter = HTMLExporter(Config())
    exporter.exclude_input_prompt=True
    exporter.exclude_output_prompt=True
    exporter.template_file = 'jekyll.tpl'
    exporter.template_path.append(str((Path('fastai_local')/'notebook').absolute()))
    return exporter

In [ ]:
def convert_nb(fname, dest_path='.'):
    "Convert a notebook `fname` to html file in `dest_path`."
    nb = read_nb(fname)
    nb['cells'] = [compose(*process_cells)(c) for c in nb['cells']]
    fname = Path(fname).absolute()
    dest_name = fname.with_suffix('.html').name.split('_')[-1]
    meta_jekyll = get_metadata(nb['cells'])
    meta_jekyll['nb_path'] = f'{fname.parent.name}/{fname.name}'
    with open(f'{dest_path}/{dest_name}','w') as f:
        f.write(_exporter().from_notebook_node(nb, resources=meta_jekyll)[0])

In [ ]:
convert_nb('99_export.ipynb')

In [ ]:
%debug

> /home/ubuntu/anaconda3/lib/python3.7/site-packages/jinja2/loaders.py(408)load()
    406             except TemplateNotFound:
    407                 pass
--> 408         raise TemplateNotFound(name)
    409 
    410     def list_templates(self):

ipdb> u
> /home/ubuntu/anaconda3/lib/python3.7/site-packages/jinja2/environment.py(804)_load_template()
    802                                          template.is_up_to_date):
    803                 return template
--> 804         template = self.loader.load(self, name, globals)
    805         if self.cache is not None:
    806             self.cache[cache_key] = template

ipdb> globals
{'range': <class 'range'>, 'dict': <class 'dict'>, 'lipsum': <function generate_lorem_ipsum at 0x7f7804220950>, 'cycler': <class 'jinja2.utils.Cycler'>, 'joiner': <class 'jinja2.utils.Joiner'>, 'namespace': <class 'jinja2.utils.Namespace'>, 'uuid4': <function uuid4 at 0x7f7877229158>}
ipdb> self
ipdb> name
'jekyll.tpl'
ipdb> u
> /home/ubuntu/anaconda3/lib

In [ ]:
(Path('/home/ubuntu/fastai_docs/dev/fastai_docs/notebook')/'jekyll.tpl').exists()

False